## T1 Feature Engineering (Clean Version)

### 1. Load Packages and Set Paths

In [1]:
import pandas as pd
import numpy as np
import os
import warnings

# Ignore warnings from pandas
warnings.filterwarnings('ignore')

# --- 📂 Step 1: Set File Paths ---
data_dir = "../DataSet/"
output_dir = "./"

# Ensure output directory exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print(f"📂 Data source directory: {data_dir}")

📂 Data source directory: ../DataSet/


### 2. Read Raw Data

In [2]:
# --- 📥 Step 2: Read Data ---
print("⏳ Reading data...")

try:
    # Reading transaction records
    txn_path = os.path.join(data_dir, 'acct_transaction.csv')
    df_txn = pd.read_csv(txn_path, dtype={'from_acct': str, 'to_acct': str})

    print(f"✅ Data reading complete! Transaction records: {df_txn.shape[0]} rows")

except FileNotFoundError as e:
    print(f"❌ Error: File not found. Please check if the path is correct.\nDetails: {e}")
    df_txn = pd.DataFrame()

⏳ Reading data...
✅ Data reading complete! Transaction records: 4435890 rows


### 3. Feature Engineering (T1 - Clean)

In [3]:
if not df_txn.empty:
    print("⚙️ Starting T1 feature engineering...")

    # Get all unique accounts
    all_accts = pd.concat([df_txn['from_acct'], df_txn['to_acct']]).unique()
    df_t1_features = pd.DataFrame({'acct': all_accts})

    # T1.1: Number of unique destination accounts transferred to
    print("   - Calculating number of unique destination accounts...")
    df_num_to = df_txn.groupby('from_acct')['to_acct'].nunique().reset_index()
    df_num_to.columns = ['acct', 'num_unique_dest_accts']
    df_t1_features = pd.merge(df_t1_features, df_num_to, on='acct', how='left')

    # T1.2: Number of unique source accounts received from
    print("   - Calculating number of unique source accounts...")
    df_num_from = df_txn.groupby('to_acct')['from_acct'].nunique().reset_index()
    df_num_from.columns = ['acct', 'num_unique_source_accts']
    df_t1_features = pd.merge(df_t1_features, df_num_from, on='acct', how='left')

    # T1.3: High frequency transactions
    print("   - Identifying high-frequency transaction accounts...")
    df_from_counts = df_txn.groupby('from_acct').size().reset_index(name='send_count')
    df_to_counts = df_txn.groupby('to_acct').size().reset_index(name='recv_count')
    df_counts = pd.merge(df_from_counts, df_to_counts, left_on='from_acct', right_on='to_acct', how='outer').fillna(0)
    df_counts['acct'] = df_counts['from_acct'].where(df_counts['from_acct'].notna(), df_counts['to_acct'])
    df_counts['total_txns'] = df_counts['send_count'] + df_counts['recv_count']
    high_freq_threshold = df_counts['total_txns'].quantile(0.95)
    print(f"   - High frequency threshold (95th percentile): {high_freq_threshold}")
    df_counts['is_high_freq'] = (df_counts['total_txns'] > high_freq_threshold).astype(int)
    df_t1_features = pd.merge(df_t1_features, df_counts[['acct', 'is_high_freq']], on='acct', how='left')

    # T1.4: 是否有外幣 (has_foreign_currency)
    print("   - Identifying accounts with foreign currency transactions...")
    if 'currency_type' in df_txn.columns:
        df_txn['is_foreign_currency_txn'] = (df_txn['currency_type'] != 'TWD').astype(int)
        foreign_accts_from = df_txn[df_txn['is_foreign_currency_txn'] == 1]['from_acct']
        foreign_accts_to = df_txn[df_txn['is_foreign_currency_txn'] == 1]['to_acct']
        foreign_accts = pd.concat([foreign_accts_from, foreign_accts_to]).unique()
        df_t1_features['has_foreign_currency'] = df_t1_features['acct'].isin(foreign_accts).astype(int)
    else:
        print("   - Warning: 'currency_type' column not found. Setting 'has_foreign_currency' to 0.")
        df_t1_features['has_foreign_currency'] = 0

    # Fill NaN values with 0 for all generated features
    df_t1_features = df_t1_features.fillna(0)

    print("✅ T1 feature engineering complete!")
    df_t1_features.head()

⚙️ Starting T1 feature engineering...
   - Calculating number of unique destination accounts...
   - Calculating number of unique source accounts...
   - Identifying high-frequency transaction accounts...
   - High frequency threshold (95th percentile): 12.0
   - Identifying accounts with foreign currency transactions...
✅ T1 feature engineering complete!


### 4. Save Processed Files

In [4]:
if 'df_t1_features' in locals():
    print("💾 Saving files...")
    output_path = os.path.join(output_dir, 'T1_features.csv')

    df_t1_features.to_csv(output_path, index=False)

    print(f"   - T1 features saved to: {output_path}")
    print("🎉 All steps completed successfully!")

💾 Saving files...
   - T1 features saved to: ./T1_features.csv
🎉 All steps completed successfully!
